# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Make a reference to the city data.csv file path
csv_path = "../weatherpy/city_df.csv"

# Import the city data.csv file as a DataFrame
city_df = pd.read_csv(csv_path, encoding="utf-8")
city_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]].astype(float)
weights = city_df["Humidity"].astype(float)

# Create a humidity heat layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


In [ ]:
# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Ideal weather conditions - max_temp less than 80 but higher than 70, Wind speed less 10 mph, and Zero cloudiness.
ideal_city = []

for city in city_df:
    if (city_df["Max Temp"] <80) and (city_df["Max Temp"]>70) and (city_df["Wind Speed"]<10) and (city_df["Cloudiness"] == 0)
    ideal_city.append(city_df['City'])


In [ ]:
#Create new dataframe fitting weather criteria
perfect_weather_df = city_df.loc[city_df["City"]==ideal_city]
perfect_weather_df.head

In [ ]:
#Drop any rows with null values from perfect_weather_df
hotel_df = perfect_weather_df.dropna(axis=0, how='all', inplace = False) 


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#find the closet hotels within a 5000 meter radious of the location
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create empty column to input hotel name
hotel_df["Hotel Name"] = ""

#Store hotel names meeting parameters as a list
#hotel_name = []

#Create parameters to pull back hotels meeting criteria
params = {
    "location": ideal_city
    "radius": "5000",
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,

# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():
    
    # get ideal city type from df
    ideal = row['City']
    
    # add keyword to params dict
    params['keyword'] = ideal

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    hotel_df.loc[index, 'Hotel Name'] = results[0]['lodging']
    
    #hotel_name.append(results)        

hotel_df.head()


In [ ]:
# add markers to heatmap
marker_locations = hotel_df['Hotel Name']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig
